In [1]:
import numpy as np
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from keras import backend, applications

config=tf.ConfigProto(device_count={
    "CPU": 4
})
backend.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


> Download the data set [HERE](https://www.kaggle.com/tongpython/cat-and-dog)

In [3]:
cwd = os.getcwd()
os.chdir(cwd)
print(os.listdir("./data/cat-and-dog"))

['test_set', 'training_set']


## Import the data and seperate it into training and testing sets

> the data is already set into training and testing directories, 8000 training and 2000 test. However we want to resize and augment these images. It will be easier to pull them in as one and execute that augmentation in one large batch

In [4]:
all_imgs = []

path = "./data/cat-and-dog/"
types = ["cats/", "dogs/"]
sets = ["training_set/", 'test_set/']

for type in types:
    for set in sets:
        for i in os.listdir(path + set + type):
            if '(' not in i and '_' not in i:
                i = path + set + type + i
                all_imgs.append(i)
  
print('All', len(all_imgs))

All 10000


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils


#may need to pip3 install opencv-python
import cv2

In [6]:
data = []
labels = []

cat_train_path = ''

for imgPath in all_imgs:
    label = imgPath.split(os.path.sep)[-2]
    
    image = cv2.imread(imgPath)
    image = cv2.resize(image, (100, 100))
    
    data.append(image)
    labels.append(label)

In [7]:
data = np.array(data, dtype='float')

data = data.reshape((data.shape[0], data.shape[1], data.shape[2], 3))

In [8]:
le = LabelEncoder()
labels = le.fit_transform(labels)

labels = np_utils.to_categorical(labels, 2)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    labels,
                                                    test_size = 0.2,
                                                    random_state=42)

In [10]:
from keras import preprocessing

datagen = preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

train_generator = datagen.flow(
    X_train,
    y_train,
    batch_size = 4)
test_generator = datagen.flow(
    X_test,
    y_test,
    batch_size = 4)

# Building our model

> I'm going to use Keras to build my model this time. If you compare the model building portion from this notebook to the CNN_tutorial.ipynb you'll see a lot of similarities. Keras is a wrapper for a few different deep learning libraries, it makes building models a little simpler by removing some of the tedious aspects of Tensorflow 

In [4]:
from keras.models import Sequential
from keras import layers
from keras.layers.core import Dropout
from keras import backend as K
from keras.optimizers import RMSprop, Adam
from keras.regularizers import l2
from keras import optimizers
from imutils import build_montages
from imutils import paths
import matplotlib.pyplot as plt
import argparse

from sklearn.metrics import classification_report

In [5]:
model = Sequential()

# FIRST LAYER
model.add(layers.Conv2D(
    filters = 32,
    kernel_size=(3, 3),
    activation='relu',
    input_shape=(150, 150, 3),
    padding='same'))

model.add(layers.MaxPooling2D((2, 2)))

#SECOND LAYER
model.add(layers.Conv2D(
    filters = 64, 
    kernel_size = (3, 3), 
    activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))

# THIRD LAYER
model.add(layers.Conv2D(
    filters = 128, 
    kernel_size = (3, 3), 
    activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))

# FOURTH LAYER
model.add(layers.Conv2D(
    filters = 128, 
    kernel_size = (3, 3), 
    activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))
model.add(Dropout(0.25))
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [6]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 73, 73, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [7]:
# This will takea very long time! 
# Also note that these models may or may not improve if we increase the number of epochs

epochs = 5

CovNet = model.fit_generator(trainData,
                             steps_per_epoch=250,
                             epochs=epochs,
                             validation_data=testData,
                             validation_steps=50,
                             verbose=1, 
                             use_multiprocessing=True)
print("Training Complete")

Epoch 1/5
 16/250 [>.............................] - ETA: 3:46 - loss: 2.3459 - acc: 0.5156

Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/home/mbruins/.conda/envs/tfdeeplearning/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/mbruins/.conda/envs/tfdeeplearning/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mbruins/.conda/envs/tfdeeplearning/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/mbruins/.conda/envs/tfdeeplearning/lib/python3.5/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/mbruins/.conda/envs/tfdeeplearning/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/mbruins/.conda/envs/tfdeeplearning/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mbruins/.conda/envs/tfde

KeyboardInterrupt: 

In [ ]:
 model.save('cats_v_dogs_5epoch_1.h5')

In [ ]:
acc = CovNet.history['acc']
val_acc = CovNet.history['val_acc']
loss = CovNet.history['loss']
val_loss = CovNet.history['val_loss']

plot_epochs = range(1, len(acc) + 1)

plt.plot(plot_epochs, acc, 'r', label="Training acc")
plt.plot(plot_epochs, val_acc, 'b', label="Validation acc")
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()
plt.plot(plot_epochs, loss, 'r', label="Training loss")
plt.plot(plot_epochs, val_loss, 'b', label = "Validation loss")
plt.title('Training and Validation Loss')
plt.legend()
plt.show()
plt.savefig('10epochs_100steps.png')

# Transfer Learning

> As you can see, our model is no better than a coin flip at differentiating between dogs and cats.


> One way to improve our model is to use a pretrained image classifier. This model has been trained on 14 million images with over 1000 different classes.

![VGG16 model](./MarkdownImages/vgg16.png)

In [14]:
import keras

imgnet_model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(100,100,3))

imgnet_model.layers.pop()

VGGmodel = Sequential(imgnet_model.layers)
for layer in VGGmodel.layers:
    layer.trainable = False

VGGmodel.add(Flatten())
VGGmodel.add(Dense(4096, activation='relu'))
VGGmodel.add(Dense(4096, activation='relu'))
VGGmodel.add(Dense(2, activation='softmax'))
VGGmodel.summary()

VGGmodel.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


58892288/58889256 [==============================] - 3s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
_________________________________________________________________
block3_conv

In [15]:
epochs = 5

VGGNet = VGGmodel.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=epochs, verbose=1)

Train on 6030 samples, validate on 2970 samples
Epoch 1/5
6030/6030 [==============================] - 327s 54ms/step - loss: 0.4052 - acc: 0.8280 - val_loss: 0.3324 - val_acc: 0.8589
Epoch 2/5
6030/6030 [==============================] - 321s 53ms/step - loss: 0.2493 - acc: 0.8952 - val_loss: 0.2990 - val_acc: 0.8717
Epoch 3/5
6030/6030 [==============================] - 351s 58ms/step - loss: 0.1668 - acc: 0.9312 - val_loss: 0.3187 - val_acc: 0.8788
Epoch 4/5
1760/6030 [=======>......................] - ETA: 2:58 - loss: 0.1203 - acc: 0.9545

KeyboardInterrupt: 